# Huggingface Token setzen

In [1]:
import os
hf_token = "hf_Token"
#hf_token = os.getenv("HF_TOKEN")

# Git Repo per HTTPs Clonen

In [ ]:
!git clone https://github.com/qvest-digital/Workshop_Agentic_AI.git

# Pfad setzen

In [2]:
#SYSTEM_PATH = "/home/simon/Workshop_Agentic_AI"
SYSTEM_PATH = "./Workshop_Agentic_AI"

# Requirements installieren

In [ ]:
!pip install -r "$SYSTEM_PATH/requirements.txt"

# Speicherfragmentierung minimieren

In [3]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Workshop Agentic AI

Dieses Notebook zeigt Schritt für Schritt, wie ein agentisches KI-System aufgebaut wird, das RAG, externe Tool-Abfragen und LLM-Reasoning kombiniert. Ziel ist es, Halluzinationen zu reduzieren, Fakten deterministisch zu beziehen und mehrere spezialisierte Sub-Modelle zu orchestrieren.

## Ablauf
- Teil 1: Lokales LLM + Halluzinationen im Griff behalten
    - Lokales LLM laden (HF-Pipeline)
    - Sampling-Verhalten & Halluzinationen untersuchen
    - Textgenerierung konfigurieren (Determinismus vs. Kreativität)
- Teil 2: Retrieval-Augmented Generation (RAG) mit dem Fantasietier Razepato
    - Texte als Embeddings in eine Vektordatenbank schreiben und vergleichen
    - Retrieval + Query + Antwortgenerierung
    - Was RAG im kontext LLM leistet
    - RAG und LLM verbinden
- Teil 3: Erlangen von Wissen durch Nutzung von MCP-Tools
    - MCP-Tools anbinden (manuell)
    - Fakten über MCP-Aufrufe beziehen (z. B. Koordinaten, Wetter, Flächen)
- Teil 4: Voll agentisches MCP – der Reiseagent denkt und handelt selbst
    - Planen & Reasoning (Chain-of-Thought, ReAct)
    - Sub-LLMs orchestrieren (Planner → Validator → Executor → Auditoren → Renderer)

# Timetable

| Zeit        | Thema                          | Inhalt / Ziel                                                |
|-------------|--------------------------------|--------------------------------------------------------------|
| 09:00–09:30 | Einchecken & Agenda vorstellen | Tagesübersicht                                               |
| 09:30–10:00 | LLM + Parameter                | HF-Setup, Tuning-Parameter, Halluzinationen => (Experimente) |
| 10:00–10:30 | Prompting (Basisprompt)        | Prompt-Engeneering, Experimente, Diskussion => (Experimente) |
| 10:30–12:00 | RAG                            | Embeddings, Vektorsuche, Retrieval, Integration              |
| 12:00–13:00 | Mittag                         | Pause                                                        |
| 13:00–13:20 | MCP-Basics                     | Konzept, Server, Call-Mechanik                               |
| 13:20–14:00 | MCP-Tool-Use                   | Fakten (Koordinaten, Wetter, Locations) über Tools           |
| 14:00–15:30 | MCP-Beispiel (manuel)          | Grundfuntkionen von MCP verstehen                            |
| 15:30–16:00 | Übergang zu Agenten            | Wie aus RAG + MCP → agentisches System wird                  |
| 16:00–16:30 | Code-Walkthrough               | Pipeline-Durchgang, Q&A                                      |
| 16:30       | Ende                           | Abschluss                                                    |

Metakommentar:
- vor jedem Block Reflexionsfragen zu Verknüpfung & Möglichkeiten.
- nach jedem Block Verbesserungsideen.

# Herstellen der Vorbedingungen aus Teil 1:

## Modell laden: lokal oder von Hugging Face

### Funktion
- Lädt ein Llama-3.1-Instruct-Modell entweder:
  - lokal aus dem Ordner ./models/llama-3.1-8b, wenn der Pfad vorhanden ist, oder
  - online von Hugging Face mit der Modell-ID meta-llama/Llama-3.1-8B-Instruct, und speichert es anschließend lokal ab.
- Die Umgebungsvariablen werden mit load_dotenv() aus einer .env-Datei geladen, u. a. das Hugging-Face-Token.

### Inputs
- Dateisystem:
  - Existenz von MODEL_PATH (./models/llama-3.1-8b).
- Umgebungsvariable:
  - HF_TOKEN (wird mit os.getenv("HF_TOKEN") gelesen) – persönliches Zugriffstoken für Hugging Face.
- Hyperparameter:
  - MODEL_ID gibt die zu ladende Modell-ID an.
- Hardware:
  - device_map="auto" versucht automatisch, GPU(s) oder CPU sinnvoll zu nutzen.
  - torch_dtype="auto" bzw. dtype="auto" lässt das Modell selbst einen sinnvollen Datentyp wählen (z. B. bfloat16 oder float16).

### Outputs
- Globale Python-Variablen:
  - tokenizer: Instanz von AutoTokenizer, konfiguriert für das Llama-3.1-Modell.
  - model: Instanz von AutoModelForCausalLM, bereit für Textgenerierung.

In [4]:
from dotenv import load_dotenv
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)

load_dotenv()

MODEL_PATH = f"{SYSTEM_PATH}/models/llama-3.1-8b"
MODEL_ID = "meta-llama/Llama-3.1-8B-Instruct"

if os.path.exists(MODEL_PATH):
    print("Lade Modell lokal …")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained( MODEL_PATH, torch_dtype="auto", device_map="auto" )

else:
    print("Lade Modell von Hugging Face …")

    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=hf_token, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(MODEL_ID, token=hf_token, dtype="auto", device_map="auto")

    # lokal speichern
    model.save_pretrained(MODEL_PATH)
    tokenizer.save_pretrained(MODEL_PATH)



Lade Modell lokal …


The tokenizer you are loading from '/home/simon/Workshop_Agentic_AI/models/llama-3.1-8b' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Pipeline erstellen
### Funktion
- Baut eine Hugging-Face-pipeline für Textgenerierung auf Basis des zuvor geladenen Modells und Tokenizers.
- Diese Pipeline kapselt:
  - Tokenisierung,
  - das Aufrufen des Modells,
  - und das Zurückkonvertieren der Token in Text.

### Inputs
- model: Causal-Language-Model (AutoModelForCausalLM), im vorherigen Block geladen.
- tokenizer: passender Tokenizer zu diesem Modell (AutoTokenizer).
- Task-Typ: "text-generation" – legt fest, dass es sich um eine generative Textaufgabe handelt.

### Outputs
- Variable:
  - llm: eine aufrufbare Pipeline-Instanz.

### Rückgabewert bei Aufruf von llm(...):
```python
[
  {
    "generated_text": "Vollständiger generierter Text (inkl. Prompt oder abhängig von den Parametern)"
  }
]
```

In [5]:
llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

Device set to use cuda:0


## Einfache Chat-Funktion zur Verfügung stellen

### Funktion
- Stellt ein vereinfachtes Chat-Interface zur Verfügung, das direkt mit einem String arbeitet.
- Die Funktion:
  - ruft intern die llm-Pipeline auf,
  - übergibt alle relevanten Generationsparameter,
  - und gibt am Ende nur den reinen generierten Text zurück (str statt verschachtelte Struktur).

### Inputs
- Pflichtparameter:
  - prompt: str – der Eingabetext an das Modell.
- Optionale Parameter zum Experimentieren
  - max_new_tokens: maximale Anzahl neu zu generierender Token.
  - temperature: steuert die Zufälligkeit (0 ≈ deterministischer, >0 zufälliger).
  - top_k: Sampling nur aus den k wahrscheinlichsten Token.
  - top_p: „Nucleus Sampling“ – Auswahl aus der kleinsten Masse der wahrscheinlichsten Token, deren Summe ≥ p ist.
  - repetition_penalty: >1.0 bestraft Wiederholungen.
  - num_beams, num_beam_groups, diversity_penalty: Parameter für Beam Search (systematisches Durchsuchen mehrerer Kandidaten).
  - early_stopping: beendet Beam Search frühzeitig, wenn bestimmte Kriterien erfüllt sind.

### Outputs
- Rückgabewert:
  - str: der vom Modell generierte Antworttext (out[0]["generated_text"] ohne führende/trailing Leerzeichen).

### Typische Verwendung:
- llama_chat("Erkläre mir kurz, was ein LLM ist.")

In späteren Zellen wird statt eines rohen Prompts ein Chat-Prompt übergeben, der mit build_chat_prompt erzeugt wird.

In [6]:
def llama_chat(
        prompt: str,
        max_new_tokens: int = 128,
        temperature: float = 0.01,
        top_k: int = 50,
        top_p: float = 1.0,
        typical_p: float = 1.0,
        repetition_penalty: float = 1.0,
        length_penalty: float = 1.0,
        no_repeat_ngram_size: int = 0,
        num_beams: int = 1,
        num_beam_groups: int = 1,
        diversity_penalty: float = 0.0,
        early_stopping: bool = False,) -> str:
    """Sehr simples Wrapper-Interface.
    Wir verwenden ein 'single prompt' Format, um es notebook-tauglich zu halten.
    """
    out = llm(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=(temperature > 0),
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        typical_p=typical_p,
        repetition_penalty=repetition_penalty,
        length_penalty=length_penalty,
        no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams,
        num_beam_groups=num_beam_groups,
        diversity_penalty=diversity_penalty,
        early_stopping=early_stopping,
        return_full_text=False,
        eos_token_id=llm.tokenizer.eos_token_id,
        pad_token_id=llm.tokenizer.pad_token_id,
    )
    return out[0]["generated_text"].strip()

Zeit zum Experimentieren (5 - 10 min.)

# Teil 3: Erlangen von Wissen durch Nutzung von MCP-Tools

Todos:
- MCP-Server starten und abfragen.
- MCP-Interaktion reglementieren (z.B. max. ein Tool, keine Chains => für Tests).
- MCP-Tool-Kette händisch durchlaufen:
    - Modell planen lassen (Tool-JSON),
    - Tool von Hand ausführen,
    - Ergebnis im Tool-Historie speichern (ähnlich wie bei RAG),
    - Modell mit derselben Frage + Tool-Historie erneut fragen.
- Das ist im Kern ein „Half-Agent“: das Planen macht das Modell, die Ausführung führen wir manuell durch
- In Teil 4 werden wie auch das automatisieren (Agentic-AI).

## MCP-Server als Subprozess starten

### Funktion
- Startet einen MCP-Server als separaten Python-Prozess über subprocess. Popen.
- Setzt PYTHONPATH so, dass mcp_server.mcp_tools.mcp.server als Modul importierbar ist.
- Hält die Projektpfade und Host/Port-Konfiguration zusammen.

### Input
- Konstanten:
    - PROJECT_ROOT / project_root: Pfad zum Projekt (SYSTEM_PATH).
    - MODULE_NAME: mcp_server.mcp_tools.mcp.server.
    - HOST, PORT: 127.0.0.1:8765.

### Umgebungsvariablen:
- Kopie von os.environ, erweitert um PYTHONPATH

### Output
- Subprozess proc:
    - MCP-Server läuft im Hintergrund unter der angegebenen PID.

### Konsole:
- Server-PID: ... zur Kontrolle, dass der Server wirklich gestartet wurde.

In [7]:
import subprocess, sys, os
from pathlib import Path

# -------------------------------------------------------------------
# Konfiguration
# -------------------------------------------------------------------
PROJECT_ROOT = Path(SYSTEM_PATH).resolve()
HOST = "127.0.0.1"
PORT = 8765
MCP_URL = f"http://{HOST}:{PORT}/mcp"

MODULE_NAME = "mcp_server_travel.mcp_tools_travel.mcp.server"

env = os.environ.copy()
env["PYTHONPATH"] = str(PROJECT_ROOT)  # falls nötig, damit mcp_tools_travel importierbar ist

In [8]:
print(f"PROJECT_ROOT: {PROJECT_ROOT}")

PROJECT_ROOT: /home/simon/Workshop_Agentic_AI


In [9]:
cmd = [
    sys.executable,
    "-m",
    MODULE_NAME,
    "--host",
    HOST,
    "--port",
    str(PORT),
]

proc = subprocess.Popen(cmd, cwd=str(PROJECT_ROOT), env=env)
print("Server-PID:", proc.pid)


Server-PID: 22901


## MCP-Tools mit Metadaten vom Server abholen

### Funktion
- Baut eine Client-Verbindung zum MCP-Server via HTTP-Transport auf.
- Führt session.initialize() aus.
- Ruft session.list_tools() auf und gibt die komplette Liste der Tools inkl. Metadaten zurück.
- Hat ein Retry-Verhalten, falls der Server noch nicht bereit ist.

### Input
- url: str = MCP_URL – z. B. http://127.0.0.1:8765/mcp.
- retries: int = 50, delay: float = 0.1 – wie oft und wie lange gewartet wird, bis der Server erreichbar ist.

### Output
- Rückgabewert von fetch_tools_with_metadata(...):
    - tools_resp.tools: Liste von Tool-Objekten (inkl. Name, Beschreibung, Input-/Output-Schema).
- Bei Fehlern nach allen Retries:
    - RuntimeError("Keine Verbindung zum MCP-Server möglich: ...").

In [10]:
import asyncio
from mcp.client.streamable_http import streamable_http_client
from mcp import ClientSession

async def fetch_tools_with_metadata(
    url: str = MCP_URL,
    retries: int = 50,
    delay: float = 0.1,
):
    last_err = None
    for _ in range(retries):
        try:
            async with streamable_http_client(url) as (read_stream, write_stream, _):
                async with ClientSession(read_stream, write_stream) as session:
                    await session.initialize()
                    tools_resp = await session.list_tools()
                    return tools_resp.tools  # komplette Objekte, nicht nur Namen
        except Exception as e:
            last_err = e
            await asyncio.sleep(delay)
    raise RuntimeError(f"Keine Verbindung zum MCP-Server möglich: {last_err!r}")

## Roh-Toolliste inspizieren

### Funktion
- Führt await fetch_tools_with_metadata() aus und speichert das Ergebnis.
- Gibt die unverarbeiteten Tool-Objekte auf der Konsole aus.

### Input
- MCP-Server muss laufen.
- Kein weiterer Parameter, MCP_URL wird aus dem globalen Kontext genommen.

### Output
- Variable tool_names_with_meta – Liste der Tool-Objekte (Client-Strukturen).

### Konsole:
- Debug-Print der Liste, z. B. <Tool name='geocode' ...> etc.

In [11]:
tool_names_with_meta = await fetch_tools_with_metadata()
print(f"tool_names_with_meta: {tool_names_with_meta}")

INFO:     127.0.0.1:59914 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:59926 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:59928 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:59932 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:59934 - "DELETE /mcp HTTP/1.1" 200 OK
tool_names_with_meta: [Tool(name='geocode', title=None, description='Geocode a destination string to coordinates using OSM Nominatim (token-free).', inputSchema={'properties': {'destination': {'title': 'Destination', 'type': 'string'}}, 'required': ['destination'], 'title': 'geocodeArguments', 'type': 'object'}, outputSchema={'description': 'Geographic coordinates in WGS84.', 'properties': {'lat': {'title': 'Lat', 'type': 'number'}, 'lon': {'title': 'Lon', 'type': 'number'}}, 'required': ['lat', 'lon'], 'title': 'Coordinates', 'type': 'object'}, icons=None, annotations=None, meta=None, execution=None), Tool(name='get_weather', title=None, description='Get a daily Open-Meteo WeatherProfile for the given c

[01/29/26 08:51:58] INFO     Created new          ]8;id=498707;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=274486;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             04eca7c00f1f406ea495                               
                             09da570b6e97                                       
                    INFO     Processing request of type            ]8;id=741877;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=154285;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.p

## Tool-Metadaten für Prompts formatieren: format_tools_for_prompt

### Funktion
- Nimmt die MCP-Tool-Objekte und erzeugt eine lesbare Textbeschreibung für das System-Prompt:
    - Name, Beschreibung,
    - JSON-Input-Schema,
    - JSON-Output-Schema.
- Ist robust genug, um sowohl Dicts als auch Objekte mit Attributen (.name, .description, .input_schema, .output_schema) zu verarbeiten.

### Input
- tools: List[Any] – Liste der Tools, z. B. aus fetch_tools_with_metadata.

### Output
- str: Formatierte Version der Tools.

In [12]:
from typing import List, Any
import json

def format_tools_for_prompt(tools: List[Any]) -> str:
    """
    Macht aus den MCP-Tool-Objekten einen lesbaren Katalog für das System-Prompt.
    Funktioniert sowohl, wenn die Tools Dicts sind, als auch, wenn sie Attribute haben.
    """
    lines = []
    for t in tools:
        is_dict = isinstance(t, dict)

        # Name
        name = getattr(t, "name", None)
        if name is None and is_dict:
            name = t.get("name")

        # Beschreibung
        desc = getattr(t, "description", None)
        if desc is None and is_dict:
            desc = t.get("description", "")
        if desc is None:
            desc = ""

        # Input-Schema
        input_schema = (
            getattr(t, "input_schema", None)
            or getattr(t, "inputSchema", None)
            or (t.get("input_schema") if is_dict else None)
            or (t.get("inputSchema") if is_dict else None)
            or {}
        )

        # Output-Schema (neu)
        output_schema = (
            getattr(t, "output_schema", None)
            or getattr(t, "outputSchema", None)
            or (t.get("output_schema") if is_dict else None)
            or (t.get("outputSchema") if is_dict else None)
            or {}
        )

        lines.append(
            f"- Name: {name}\n"
            f"  Beschreibung: {desc}\n"
            f"  Eingabe-Schema (JSON): {json.dumps(input_schema, ensure_ascii=False)}\n"
            f"  Ausgabe-Schema (JSON): {json.dumps(output_schema, ensure_ascii=False)}"
        )

    return "\n\n".join(lines)

## Formatierten Tool-Katalog ansehen

### Funktion
- Ruft format_tools_for_prompt(tool_names_with_meta) auf.
- Gibt den formatierten Katalog auf der Konsole aus, damit du siehst, welche Tools der Server anbietet.

### Input
- tool_names_with_meta aus dem vorherigen Schritt.

### Output
- tool_names_with_meta_formated: str – fertiger Tool-Katalog-String.

### Konsolenausgabe
- Konsolenausgabe des Strings (für menschlichen Check).

In [13]:
tool_names_with_meta_formated = format_tools_for_prompt(tool_names_with_meta)
print(tool_names_with_meta_formated)

- Name: geocode
  Beschreibung: Geocode a destination string to coordinates using OSM Nominatim (token-free).
  Eingabe-Schema (JSON): {"properties": {"destination": {"title": "Destination", "type": "string"}}, "required": ["destination"], "title": "geocodeArguments", "type": "object"}
  Ausgabe-Schema (JSON): {"description": "Geographic coordinates in WGS84.", "properties": {"lat": {"title": "Lat", "type": "number"}, "lon": {"title": "Lon", "type": "number"}}, "required": ["lat", "lon"], "title": "Coordinates", "type": "object"}

- Name: get_weather
  Beschreibung: Get a daily Open-Meteo WeatherProfile for the given coordinates and date range (YYYY-MM-DD).
  Eingabe-Schema (JSON): {"properties": {"lat": {"title": "Lat", "type": "number"}, "lon": {"title": "Lon", "type": "number"}, "start_date": {"title": "Start Date", "type": "string"}, "end_date": {"title": "End Date", "type": "string"}, "include_raw": {"default": false, "title": "Include Raw", "type": "boolean"}}, "required": ["lat"

## System-Prompt für „manuelles“ MCP: build_tool_system_prompt

### Funktion
- Baut einen großen System-Prompt, der:
    - Rolle: Reiseplaner mit RAG + MCP-Tools.
    - Nutzungsregeln für Kontext (RAG) beschreibt.
    - Strenge Regeln für MCP-Tool-Nutzung definiert:
        - max. ein Tool pro Benutzernachricht,
        - keine Tool-Ketten in einer Antwort,
        - keine geratenen Argumente,
        - nur Argumente nutzen, die explizit im Verlauf stehen.
    - Ein genaues JSON-Format für Tool-Aufrufe vorschreibt.
- Der Tool-Katalog (format_tools_for_prompt) wird am Ende eingebettet.

### Input
- tools: List[Any] – die vom MCP-Server gelieferten Tools.

### Output
- str: system_prompt_for_tools_and_rag – System-Prompt mit:
    - RAG-Kontextnutzung,
    - MCP-Toolnutzung,
    - Unsicherheits-Handling,
    - Ablehnungsfällen,
    - Protokoll für JSON-Tool-Calls (json {...} ).
    - Wird später an build_chat_prompt_with_rag_and_tools übergeben.

In [14]:
from typing import Any, List

def build_tool_system_prompt(tools: List[Any]) -> str:
    tool_catalog = format_tools_for_prompt(tools)
    return f"""
Du bist ein persönlicher Reiseplaner.

Deine Aufgaben:
- Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.
- Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).
- Dir stehen MCP-Tools zur Verfügung, um Informationen zu beschaffen, die nicht in deinem Weltwissen vorhanden sind.

Nutzung von Kontext (RAG):
- Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.
- Du darfst alle darin enthaltenen Fakten verwenden.
- Wenn eine Information im Kontext steht, darfst du sie verwenden, auch wenn du sie nicht aus deinem allgemeinen Weltwissen kennst.

Nutzung von MCP-Tools:
- Du darfst pro Benutzernachricht höchstens EIN Tool aufrufen.
- Du darfst KEINE Tool-Ketten planen (kein: „erst Tool A, dann Tool B“ innerhalb derselben Antwort).
- Der Nutzer kann dich aber mehrfach aufrufen, um Toolketten über mehrere Nachrichten zu simulieren.
- Du darfst ein Tool NUR dann aufrufen, wenn **alle** benötigten Argumente explizit vorliegen, und zwar entweder:
  (a) direkt im Text der aktuellen Benutzernachricht oder
  (b) in klar lesbaren Ergebnissen früherer Tool-Aufrufe, die in der Tool-History sichtbar sind (z.B. gespeicherte Koordinaten, IDs oder Datumsangaben).
- Du DARFST KEINE Tool-Argumente aus deinem allgemeinen Weltwissen ableiten oder raten.
- Verwende für Tool-Argumente nur Informationen, die explizit im Gesprächstext oder in der Tool-Historie stehen.
- Wenn zur Beantwortung einer Frage mehrere neue Tool-Aufrufe in Folge nötig wären und der Nutzer die notwendigen Vor-Tools nicht bereits zuvor aufgerufen hat, dann darfst du KEIN Tool benutzen und musst erklären, dass das aktuell nicht unterstützt wird.
- Wenn ein Tool bereits mit bestimmten Argumenten aufgerufen wurde UND das Ergebnis im Verlauf steht, darfst du dieses Ergebnis wie Weltwissen verwenden und sollst das Tool mit denselben Argumenten nicht noch einmal aufrufen.

Tools:
{tool_catalog}

Protokoll für Tool-Aufrufe:
- Wenn du KEIN Tool benötigst oder benutzen darfst, antworte ganz normal im Fließtext.
- Wenn du EIN Tool aufrufen willst, antworte NICHT im Fließtext, sondern ausschließlich mit einem einzigen JSON-Objekt (kein Markdown, keine ```-Blöcke, keine zusätzlichen Worte) der Form:

  ```json
  {{
    "tool": "<tool_name>",
    "arguments": {{
      "argument1": <Wert>,
      "argument2": <Wert>
    }}
  }}
  ```

- Das JSON muss direkt mit ```json beginnen und mit ``` enden, ohne einleitenden oder nachfolgenden Text.
- <tool_name> muss mit einem der oben aufgeführten Namen übereinstimmen.
- "arguments" muss genau zu dem Eingabe-Schema des jeweiligen Tools passen.
- Füge KEINE zusätzlichen Felder hinzu.
- Wenn du auf Basis des bisherigen Gesprächs und der Tool-Historie bereits alle nötigen Informationen hast, um die Frage zu beantworten, rufe KEIN Tool auf, sondern gib direkt eine inhaltliche Antwort im Fließtext.

Umgang mit Wissen und Unsicherheit:
- Erfinde keine Fakten und spekuliere nicht.
- Wenn eine Information weder in deinem Weltwissen noch im bereitgestellten Kontext vorkommt und auch nicht in der MCP-Tool-Historie zu finden ist oder per MCP-Tool-Call erlangt werden kann, weise darauf hin.
- Falls dir wirklich Informationen fehlen, formuliere eine normale, erklärende Antwort und beschreibe, was du stattdessen sicher sagen kannst.

Einschränkungen (Ablehnungsfälle):
- Lehne Anfragen ab, die eindeutig NICHT zum Reise-/Tourismuskontext gehören.

Verwende beim Ablehnen folgenden Text:
„Diese Frage kann ich nicht beantworten, da sie Informationen erfordert, die ich nicht verlässlich bereitstellen kann.“

Verwende bei Themen, die klar nicht zum Reise-Kontext gehören:
„Diese Frage kann ich nicht beantworten, da sie nicht dem Kontext des Assistenten entspricht.“
"""

## System-Prompt generieren und inspizieren

### Funktion
- Ruft build_tool_system_prompt(tool_names_with_meta) auf.
- Druckt den resultierenden System-Prompt.

### Input
- tool_names_with_meta: Toolliste vom Server.

### Output
- system_prompt_for_tools_and_rag: str – vollständig gebauter Prompt.

### Konsole:
- Ausgabe des gesamten Prompts (zur Überprüfung, ob alles wie gewünscht im Text steht).

In [15]:
system_prompt_for_tools_and_rag = build_tool_system_prompt(tool_names_with_meta)
print(system_prompt_for_tools_and_rag)


Du bist ein persönlicher Reiseplaner.

Deine Aufgaben:
- Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.
- Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).
- Dir stehen MCP-Tools zur Verfügung, um Informationen zu beschaffen, die nicht in deinem Weltwissen vorhanden sind.

Nutzung von Kontext (RAG):
- Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.
- Du darfst alle darin enthaltenen Fakten verwenden.
- Wenn eine Information im Kontext steht, darfst du sie verwenden, auch wenn du sie nicht aus deinem allgemeinen Weltwissen kennst.

Nutzung von MCP-Tools:
- Du darfst pro Benutzernachricht höchstens EIN Tool aufrufen.
- Du darfst KEINE Tool-Ketten planen (kein: „erst Tool A, dann Tool B“ innerhalb derselben Antwort).
- Der Nutzer kann dich aber mehrfach aufrufen, um

## Chat-Prompt-Builder für RAG + Tools: build_chat_prompt_with_rag_and_tools

### Funktion
- Generalisiert den bisherigen Prompt-Builder:
    - System-Prompt (mit MCP-Tool-Regeln),
    - optionaler Override für finales Ergebniss => „keine Tools erlaubt“,
    - normale Chat-History,
    - MCP-Tool-History als eigener System-Block,
    - RAG-Kontext als separater System-Block,
    - aktuelle User-Nachricht.
- Das Ergebnis wird durch das Llama-Chat-Template gejagt → finaler Promptstring.

### Input
- system_prompt: Optional[str] – z. B. system_prompt_for_tools_and_rag.
- user_prompt: str – aktuelle Benutzernachricht.
- history: Optional[List[Tuple[str, str]]] – Dialoghistorie (User/Assistant-Paare).
- tool_history: Optional[List[Dict[str, Any]]] – Liste von Tool-Aufrufen:
    - z. B. {"tool": "geocode", "arguments": {...}, "result": {...}}.
- allow_tools: bool = True:
    - Wenn False, wird ein zusätzlicher System-Hinweis eingefügt, dass das Modell keine Tools aufrufen darf und nur Fließtext liefern soll.

### Output
- prompt: str – Chat-Prompt im Template-Format, der folgende Komponenten enthalten kann:
    - system: Tool- und RAG-Regeln,
    - system: optional Tool-Verbot,
    - system: Tool-Historie,
    - system: RAG-Kontext,
    - user: aktuelle Frage.
- Dieser String ist direkt Eingabe für llama_chat.

In [16]:
from typing import List, Dict, Optional, Tuple, Any

def build_chat_prompt_with_rag_and_tools(
    system_prompt: Optional[str],
    user_prompt: str,
    history: Optional[List[Tuple[str, str]]] = None,
    tool_history: Optional[List[Dict[str, Any]]] = None,
    allow_tools: bool = True,
) -> str:
    messages: List[Dict[str, str]] = []

    rag_context = ""

    # 1) System-Prompt
    if system_prompt:
        messages.append(
            {
                "role": "system",
                "content": system_prompt,
            }
        )

    # 1b) Override für Phase 2:
    if not allow_tools:
        messages.append(
            {
                "role": "system",
                "content": (
                    "WICHTIG: In dieser Runde darfst du KEINE MCP-Tools aufrufen. "
                    "Verwende ausschließlich die bereits im Verlauf vorhandenen Informationen, "
                    "einschließlich der Ergebnisse früherer Tool-Aufrufe. "
                    "Antworte im normalen Fließtext. "
                    "Gib KEIN JSON und KEINE ```json-Codeblöcke aus."
                ),
            }
        )

    # 2) Verlauf (User/Assistant)
    if history:
        for user_msg, assistant_msg in history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})

    # 3) Tool-History (falls vorhanden)
    if tool_history:
        messages.append({
            "role": "system",
            "content": (
                "Tool-Historie (sichtbar für dich, verwende sie bei Bedarf für Parameter):\n"
                + json.dumps(tool_history, ensure_ascii=False)
            )
        })

    # 4) RAG-Kontext
    if rag_context:
        messages.append(
            {
                "role": "system",
                "content": (
                    "Das folgende ist Kontext aus einer Wissensdatenbank. "
                    "Er ist nicht vom User. Nutze ihn nur, wenn er relevant ist:\n\n"
                    f"{rag_context}"
                ),
            }
        )

    # 5) aktuelle User-Nachricht
    messages.append({"role": "user", "content": user_prompt})

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    return prompt

## Test der Anweisungen (z.B. Toolkettenverbot)

### Funktion
- Testet, was passiert, wenn der User eine Anfrage stellt, die eigentlich eine Tool-Kette erfordert:
    - „Ich möchte eine Reise nach Barcelona machen. Wie ist heute (YYYY-MM-DD) das Wetter in Barcelona?“

### Tools:
- Um korrekt zu antworten, bräuchte man geocode → get_weather.
- Der System-Prompt verbietet aber Toolketten.

### Input
- today = date.today() – aktuelles Datum.
- user_input: Frage mit Datum.
- history = [], tool_history=None.
- build_chat_prompt_with_rag_and_tools(..., allow_tools=True).

### Output
- prompt: str – LLM-Prompt (inkl. Toolregeln).
- assistant_text: Output von llama_chat.
    - Erwartet:
        - entweder Ablehnung („kann ich nicht, da mehrere Tools nötig“),
        - oder Erklärung, dass das nicht in einem Schritt geht,
        - aber kein Tool-Call, der beide Tools in einer Antwort kombiniert.

### Konsole:
- Input (Prompt) + Output (Antwort).

In [17]:
from datetime import date

# Returns the current local date
today = date.today()

user_input = f"Ich möchte eine Reise nach Barcelona machen. Wie ist heute ({today}) das Wetter in Barcelona?"

history = []

prompt = build_chat_prompt_with_rag_and_tools(
    system_prompt=system_prompt_for_tools_and_rag,
    user_prompt=user_input,
    history=history,
    tool_history=None,
)

print(f"Input: {prompt}\n\n")
assistant_text = llama_chat(prompt, max_new_tokens=1024)
print(f"Output: {assistant_text}\n\n")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Input: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Du bist ein persönlicher Reiseplaner.

Deine Aufgaben:
- Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.
- Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).
- Dir stehen MCP-Tools zur Verfügung, um Informationen zu beschaffen, die nicht in deinem Weltwissen vorhanden sind.

Nutzung von Kontext (RAG):
- Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.
- Du darfst alle darin enthaltenen Fakten verwenden.
- Wenn eine Information im Kontext steht, darfst du sie verwenden, auch wenn du sie nicht aus deinem allgemeinen Weltwissen kennst.

Nutzung von MCP-Tools:
- Du darfst pro Benutzernachricht höchstens EIN Tool aufrufen.
- Du darfst KEINE To

## Toolplanung bei erlaubten Aufrufen testen (z.B. Geokoordinaten von Barcelona)

### Funktion
- Neue Userfrage, jetzt nur nach Geokoordinaten:
    - „Wie sind die genauen Geokoordinaten von Barcelona?“
- System darf pro Nachricht ein Tool nutzen → erwartetes Verhalten:
    - Modell plant, geocode mit destination="Barcelona" aufzurufen und gibt den entsprechenden JSON-Tool-Call aus.

### Input
- user_input = "...Geokoordinaten von Barcelona?"
- history = [], tool_history=None.
- Promptbau mit allow_tools=True.

### Output
- assistant_text:
- idealerweise ```json {"tool": "geocode", "arguments": {"destination": "Barcelona"}} ```.

### Konsole:
- wieder Prompt + Modelloutput zur Kontrolle.

In [18]:
user_input = "Ich möchte eine Reise nach Barcelona machen. Wie sind die genauen Geokoordinaten von Barcelona?"

history = []

prompt = build_chat_prompt_with_rag_and_tools(
    system_prompt=system_prompt_for_tools_and_rag,
    user_prompt=user_input,
    history=history,
    tool_history=None,
)

print(f"Input: {prompt}\n\n")
assistant_text = llama_chat(prompt, max_new_tokens=512)
print(f"Output: {assistant_text}\n\n")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Input: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Du bist ein persönlicher Reiseplaner.

Deine Aufgaben:
- Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.
- Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).
- Dir stehen MCP-Tools zur Verfügung, um Informationen zu beschaffen, die nicht in deinem Weltwissen vorhanden sind.

Nutzung von Kontext (RAG):
- Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.
- Du darfst alle darin enthaltenen Fakten verwenden.
- Wenn eine Information im Kontext steht, darfst du sie verwenden, auch wenn du sie nicht aus deinem allgemeinen Weltwissen kennst.

Nutzung von MCP-Tools:
- Du darfst pro Benutzernachricht höchstens EIN Tool aufrufen.
- Du darfst KEINE To

## Tool-Call aus dem Modelloutput extrahieren: try_parse_tool_call

### Funktion
- Nimmt den rohen Modelloutput und versucht, einen Tool-Call im JSON-Format zu extrahieren.
- Debug Unterstützt:
    - plain String,
    - Listen/Dictionaries mit Feldern wie generated_text oder text,
    - json ... -Codeblöcke,
    - reinen JSON-Text,
    - „Brute Force“: alle {...}-Blöcke im Text durchprobieren.
- Extrahiert - wenn möglich - ein JSON.
- Valid tool call: Dict mit {"tool": <str>, "arguments": <dict>}.

### Input
- model_output: Any – Output von llama_chat oder Pipeline.
- debug: bool = False – bei True detaillierte Debugprints.

### Output
- Entweder:
    - {"tool": "<tool_name>", "arguments": {...}}
    - oder None, falls kein gültiger Tool-Call gefunden wird.
- Das Ergebnis wird z. B. in parsed_call gespeichert.

In [19]:
import re
from typing import Any, Dict, Optional

def try_parse_tool_call(model_output: Any, debug: bool = False) -> Optional[Dict[str, Any]]:
    """
    Versucht, einen Tool-Call im JSON-Format im Modell-Output zu finden.
    Erwartetes JSON-Format:

        {
          "tool": "<tool_name>",
          "arguments": { ... }
        }

    Rückgabe:
        {"tool": str, "arguments": dict} oder None.
    """

    if debug:
        print("=== try_parse_tool_call: RAW model_output ===")
        print("type(model_output):", type(model_output))
        print("repr(model_output):", repr(model_output))
        print("===========================================\n")

    # 1) Auf String normalisieren
    text: str

    if isinstance(model_output, str):
        text = model_output
        if debug:
            print(">> Interpretiere model_output als einfachen String.\n")
    elif isinstance(model_output, list):
        if debug:
            print(">> model_output ist eine Liste, Länge:", len(model_output))
        if len(model_output) > 0 and isinstance(model_output[0], dict):
            first = model_output[0]
            if debug:
                print(">> Erstes Element der Liste ist ein Dict, keys:", list(first.keys()))
            if "generated_text" in first:
                text = first["generated_text"]
                if debug:
                    print('>> Nutze first["generated_text"] als Text.\n')
            elif "text" in first:
                text = first["text"]
                if debug:
                    print('>> Nutze first["text"] als Text.\n')
            else:
                text = str(model_output)
                if debug:
                    print(">> Keine 'generated_text'/'text'-Keys gefunden, fallback = str(model_output).\n")
        else:
            text = str(model_output)
            if debug:
                print(">> Liste ohne Dict als erstes Element. Fallback = str(model_output).\n")
    elif isinstance(model_output, dict):
        if debug:
            print(">> model_output ist ein Dict, keys:", list(model_output.keys()))
        if "generated_text" in model_output:
            text = model_output["generated_text"]
            if debug:
                print('>> Nutze model_output["generated_text"] als Text.\n')
        elif "text" in model_output:
            text = model_output["text"]
            if debug:
                print('>> Nutze model_output["text"] als Text.\n')
        else:
            text = str(model_output)
            if debug:
                print(">> Keine 'generated_text'/'text'-Keys, fallback = str(model_output).\n")
    else:
        text = str(model_output)
        if debug:
            print(">> model_output ist weder str, list noch dict. Fallback = str(model_output).\n")

    text = text.strip()

    if debug:
        print("=== Normalisierter Text ===")
        print(text)
        print("===========================================\n")

    # 2) Speziell: JSON in ```json ... ```-Codeblock finden
    code_block_match = re.search(r"```json\s*(.*?)```", text, re.DOTALL | re.IGNORECASE)
    if code_block_match:
        json_candidate = code_block_match.group(1).strip()
        if debug:
            print(">> Finde ```json```-Codeblock, versuche diesen Inhalt als JSON zu parsen:")
            print(json_candidate)
            print("-------------------------------------------\n")

        try:
            data = json.loads(json_candidate)
            if isinstance(data, dict) and isinstance(data.get("tool"), str) and isinstance(data.get("arguments"), dict):
                if debug:
                    print(">> Codeblock ist gültiges JSON mit tool/arguments. Erfolg!\n")
                return {"tool": data["tool"], "arguments": data["arguments"]}
            else:
                if debug:
                    print(">> Codeblock ist JSON, aber kein passendes tool/arguments-Objekt.\n")
        except json.JSONDecodeError as e:
            if debug:
                print(">> Codeblock ist kein valides JSON:", e, "\n")

    # 3) Volltext-JSON als Versuch (falls Modell wirklich nur JSON ausgibt)
    if debug:
        print(">> Versuche, den gesamten Text als JSON zu parsen...\n")
    try:
        data = json.loads(text)
        if isinstance(data, dict) and isinstance(data.get("tool"), str) and isinstance(data.get("arguments"), dict):
            if debug:
                print(">> Volltext ist gültiges JSON mit tool/arguments. Erfolg!\n")
            return {"tool": data["tool"], "arguments": data["arguments"]}
        else:
            if debug:
                print(">> Volltext ist JSON, aber kein tool/arguments-Objekt.\n")
    except json.JSONDecodeError:
        if debug:
            print(">> Volltext ist kein valides JSON, nutze Brute-Force-Suche nach JSON-Blöcken...\n")

    # 4) Brute-Force: für jede '{' alle möglichen '}'-Enden testen
    if debug:
        print(">> Starte Brute-Force-Suche nach JSON-Blöcken...\n")

    s = text
    n = len(s)
    brace_positions = [i for i, ch in enumerate(s) if ch == "{"]

    if debug:
        print(">> Anzahl '{'-Positionen:", len(brace_positions))

    for start in brace_positions:
        for end in range(start + 1, n):
            if s[end] == "}":
                candidate = s[start:end + 1]
                candidate_stripped = candidate.strip()
                # kleine Heuristik: mindestens 'tool' oder 'arguments' sollten drinstehen, sonst sparen wir uns json.loads
                if ("tool" not in candidate_stripped) and ("arguments" not in candidate_stripped):
                    continue

                if debug:
                    print(">>> Teste Kandidat (start={}, end={}):".format(start, end))
                    print(candidate_stripped)
                try:
                    data = json.loads(candidate_stripped)
                except json.JSONDecodeError:
                    if debug:
                        print(">>> Kandidat ist kein valides JSON.\n")
                    continue

                if not isinstance(data, dict):
                    if debug:
                        print(">>> Kandidat ist JSON, aber kein Dict.\n")
                    continue

                tool_name = data.get("tool")
                arguments = data.get("arguments")

                if isinstance(tool_name, str) and isinstance(arguments, dict):
                    if debug:
                        print(">>> Kandidat ist gültiger Tool-Call!")
                        print(f"    tool={tool_name}, arguments={arguments}\n")
                    return {"tool": tool_name, "arguments": arguments}

    if debug:
        print(">> Kein valider Tool-Call gefunden. Rückgabe = None.\n")

    return None

## Tool-Call parsen und inspizieren

### Funktion
- Ruft try_parse_tool_call(assistant_text) auf.
- Druckt den geparsten Tool-Call.

### Input
- assistant_text – Modellantwort aus dem „Koordinaten von Barcelona“-Run.

### Output
- parsed_call: {"tool": "...", "arguments": {...}} oder None.
- Konsolenausgabe des Ergebnisses.

In [20]:
parsed_call = try_parse_tool_call(assistant_text)
print(parsed_call)

{'tool': 'geocode', 'arguments': {'destination': 'Barcelona'}}


## Ein MCP-Tool tatsächlich ausführen: call_mcp_tool_once

### Funktion
- Führt genau ein MCP-Tool aus und gibt ein „normales“ Python-Objekt zurück.
- Kapselt:
    - Aufbau der HTTP-Clientverbindung,
    - session.initialize(),
    - session.call_tool(tool_name, arguments).

### Input
- tool_name: str – z. B. "geocode".
- arguments: Dict[str, Any] – z. B. {"destination": "Barcelona"}.
- url: str = MCP_URL.

### Output
- Rückgabewert:
    - Primär: resp.structuredContent, falls vorhanden (oft schon ein Dict).
    - Sekundär: json.loads(content[0].text), wenn es Text-JSON gibt.
    - Fallback: ein einfaches Dict mit Metadaten (meta, isError) oder Text.
- Typischer Output (Dict):
    - {"lat": 41.38..., "lon": 2.17...}.

In [21]:
async def call_mcp_tool_once(
    tool_name: str,
    arguments: Dict[str, Any],
    url: str = MCP_URL,
) -> Any:
    """
    Führt genau EIN MCP-Tool aus und gibt ein "normales" Python-Objekt zurück
    (idealerweise ein dict), das direkt JSON-serialisierbar ist.
    """
    async with streamable_http_client(url) as (read_stream, write_stream, _):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            resp = await session.call_tool(tool_name, arguments)

            # Priorität 1: structuredContent (das ist bei dir schon ein dict)
            structured = getattr(resp, "structuredContent", None)
            if structured is not None:
                return structured

            # Priorität 2: content[0].text als JSON parsen, falls vorhanden
            content = getattr(resp, "content", None)
            if (
                content
                and isinstance(content, list)
                and hasattr(content[0], "text")
                and isinstance(content[0].text, str)
            ):
                txt = content[0].text
                try:
                    return json.loads(txt)
                except json.JSONDecodeError:
                    # Falls es kein JSON ist – dann geben wir einfach den Text zurück
                    return {"text": txt}

            # Fallback: zur Not alles in ein dict packen
            return {
                "meta": getattr(resp, "meta", None),
                "isError": getattr(resp, "isError", None),
            }


## Tool-Aufruf wirklich ausführen (Geocode Barcelona)

### Funktion
- Verwendet das geparste parsed_call, um das passende MCP-Tool auszuführen.
- Speichert den Tool-Output in tool_result.

### Input
- parsed_call["tool"], parsed_call["arguments"].
- MCP-Server muss laufen.

### Output
- tool_result – Ergebnisobjekt, z. B. Koordinaten für Barcelona.

### Konsole:
- tool_result: {...} zur Sichtkontrolle.

In [22]:
tool_name = parsed_call["tool"]
arguments = parsed_call["arguments"]

tool_result = await call_mcp_tool_once(tool_name, arguments)


print(f"tool_result: {tool_result}")

INFO:     127.0.0.1:41046 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:41062 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:41070 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:41078 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:41090 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:41096 - "DELETE /mcp HTTP/1.1" 200 OK
tool_result: {'lat': 41.3825802, 'lon': 2.177073}


[01/29/26 08:53:05] INFO     Created new          ]8;id=162599;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=892428;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             414c7d48c3134594805e                               
                             e4a64a9094ef                                       
                    INFO     Processing request of type            ]8;id=68032;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=888612;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py

## Tool-Historie initialisieren und füllen

### Funktion
- Legt eine Liste tool_history an.
- Hängt den eben ausgeführten Tool-Call samt Ergebnis daran an.
- Das ist die „Tool-Gedächtnisstruktur“ für spätere Prompts.

### Input
- tool_name, arguments, tool_result.

### Output
- tool_history: List[Dict[str, Any]]

### Konsole:
- print(tool_history).

In [23]:
# Tool-Verlauf speichern
tool_history = []

tool_history.append(
    {
        "tool": tool_name,
        "arguments": arguments,
        "result": tool_result,  # z.B. {'lat': 41.8933203, 'lon': 12.4829321}
    }
)

print(tool_history)

[{'tool': 'geocode', 'arguments': {'destination': 'Barcelona'}, 'result': {'lat': 41.3825802, 'lon': 2.177073}}]


## Koordinaten aus Tool-Historie nutzen, um die Nutzeranfrage final zu Beantworten

### Funktion
- Zeigt, wie das LLM nun auf Basis des bisherigen Tool-Ergebnisses antwortet.
- System: Tools sind erlaubt – aber allow_tools=False fügt einen Override-Systemprompt ein:
    - „Du darfst KEINE MCP-Tools aufrufen...“
- Da nun bereits alle notwendigen Informationen aus der MCP-Abfrage vorliegen

### Input
- user_input: wieder „Wie sind die genauen Geokoordinaten von Barcelona?“
- tool_history: enthält den geocode-Call + Result.
- allow_tools=False.

### Output
- assistant_text: LLM soll jetzt:
    - keine Tools mehr aufrufen,
    - die Koordinaten aus tool_history direkt als „Wissen“ verwenden
    - und im Fließtext antworten (z. B. „Barcelona liegt bei ca. Breitengrad X, Längengrad Y“).

### Konsole:
- Prompt + Antwort.

In [24]:
user_input = "Ich möchte eine Reise nach Barcelona machen. Wie sind die genauen Geokoordinaten von Barcelona?"

prompt = build_chat_prompt_with_rag_and_tools(
    system_prompt=system_prompt_for_tools_and_rag,
    user_prompt=user_input,
    history=history,
    tool_history=tool_history,
    allow_tools=False,
)

print(f"Input: {prompt}\n\n")
print("Ende input\n\n")
assistant_text = llama_chat(prompt, max_new_tokens=512)
print(f"Output: {assistant_text}\n\n")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Input: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Du bist ein persönlicher Reiseplaner.

Deine Aufgaben:
- Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.
- Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).
- Dir stehen MCP-Tools zur Verfügung, um Informationen zu beschaffen, die nicht in deinem Weltwissen vorhanden sind.

Nutzung von Kontext (RAG):
- Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.
- Du darfst alle darin enthaltenen Fakten verwenden.
- Wenn eine Information im Kontext steht, darfst du sie verwenden, auch wenn du sie nicht aus deinem allgemeinen Weltwissen kennst.

Nutzung von MCP-Tools:
- Du darfst pro Benutzernachricht höchstens EIN Tool aufrufen.
- Du darfst KEINE To

## Erneute Wetteranfrage basierend auf dem zuvor erlangten Wissen der Koordinaten

### Funktion
- Jetzt kombinieren der Toolcalls von Hand:
    - Koordinaten von Barcelona sind bereits in tool_history.
    - User fragt erneut nach dem heutigen Wetter in Barcelona.
    - Da alle Argumente für get_weather nun vorhanden sind, darf das Modell ein Tool aufrufen.

### Input
- today = date.today()
- user_input: „Wie ist heute (YYYY-MM-DD) das Wetter in Barcelona? Nutze die Geokoordinaten von vorhin…“
- tool_history: enthält geocode-Resultat aus vorherigem Call:
- allow_tools=True.

### Output
- assistant_text: sollte jetzt einen JSON-Tool-Call für get_weather produzieren.

### Konsole:
- Prompt + Modelloutput.

In [25]:
from datetime import date

# Returns the current local date
today = date.today()

user_input = f"Ich möchte eine Reise nach Barcelona machen. Wie ist heute ({today}) das Wetter in Barcelona? Nutze die Geokoordinaten von vorhin um das Wetter abzufragen."

prompt = build_chat_prompt_with_rag_and_tools(
    system_prompt=system_prompt_for_tools_and_rag,
    user_prompt=user_input,
    history=history,
    tool_history=tool_history,
)

print(f"Input: {prompt}\n\n")
assistant_text = llama_chat(prompt, max_new_tokens=512)
print(f"Output: {assistant_text}\n\n")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Input: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Du bist ein persönlicher Reiseplaner.

Deine Aufgaben:
- Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.
- Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).
- Dir stehen MCP-Tools zur Verfügung, um Informationen zu beschaffen, die nicht in deinem Weltwissen vorhanden sind.

Nutzung von Kontext (RAG):
- Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.
- Du darfst alle darin enthaltenen Fakten verwenden.
- Wenn eine Information im Kontext steht, darfst du sie verwenden, auch wenn du sie nicht aus deinem allgemeinen Weltwissen kennst.

Nutzung von MCP-Tools:
- Du darfst pro Benutzernachricht höchstens EIN Tool aufrufen.
- Du darfst KEINE To

## Wetter-Tool ausführen und finale Antwort generieren

### Funktion
- Nächste manuelle Agentenrunde:
    - parsed_call = try_parse_tool_call(assistant_text) → Wetter-Tool-Call extrahieren.
    - tool_result = await call_mcp_tool_once(tool_name, arguments) → Wetterdaten holen.
    - tool_history.append(...) → Wetter-Resultat zur Historie hinzufügen.
     - Nochmals build_chat_prompt_with_rag_and_tools(..., allow_tools=False) mit denselben user_input + aktualisierter tool_history.
    - llama_chat(...) → jetzt Endantwort ohne neue Tools, aber auf Basis der gespeicherten Wetterdaten.

### Input
- assistant_text (mit Wetter-Tool-Call).
- MCP-Server, tool_history (mit geocode), user_input (Wetterfrage).
- allow_tools=False beim zweiten Prompt.

### Output
- parsed_call: Wetter-Tool-Call.
- tool_result: Wetterprofil (Open-Meteo Profil, z. B. Temperaturen, Niederschlag).
- Aktualisierte tool_history: enthält jetzt zwei Einträge (geocode, get_weather).
- assistant_text (zweiter LLM-Call):
- Modell erklärt das Wetter in Barcelona am heutigen Datum,
- ohne weitere Tools aufzurufen,
- und kann auf die strukturierten Wetterdaten zurückgreifen.

In [26]:
parsed_call = try_parse_tool_call(assistant_text)
print(parsed_call)

tool_name = parsed_call["tool"]
arguments = parsed_call["arguments"]

tool_result = await call_mcp_tool_once(tool_name, arguments)


print(f"tool_result: {tool_result}")

tool_history.append(
    {
        "tool": tool_name,
        "arguments": arguments,
        "result": tool_result,  # z.B. {'lat': 41.8933203, 'lon': 12.4829321}
    }
)

print(tool_history)

prompt = build_chat_prompt_with_rag_and_tools(
    system_prompt=system_prompt_for_tools_and_rag,
    user_prompt=user_input,
    history=history,
    tool_history=tool_history,
    allow_tools=False,
)

print(f"Input: {prompt}\n\n")
print("Ende input\n\n")
assistant_text = llama_chat(prompt, max_new_tokens=512)
print(f"Output: {assistant_text}\n\n")

[01/29/26 08:54:06] INFO     Created new          ]8;id=539661;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=64168;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             1338817f758e4f2292d4                               
                             b08213dccaa3                                       
                    INFO     Processing request of type            ]8;id=551109;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=394592;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py

{'tool': 'get_weather', 'arguments': {'lat': 41.3825802, 'lon': 2.177073, 'start_date': '2026-01-29', 'end_date': '2026-01-29', 'include_raw': False}}
INFO:     127.0.0.1:52426 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:52442 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:52454 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:52468 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:52478 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:52490 - "DELETE /mcp HTTP/1.1" 200 OK
tool_result: {'start_date': '2026-01-29', 'end_date': '2026-01-29', 'days': [{'date': '2026-01-29', 'temp_min_c': 6.9, 'temp_max_c': 13.9, 'precipitation_mm': 0.0, 'wind_max_kmh': 13.3, 'weather_code': 3}], 'temp_min_c': None, 'temp_max_c': None, 'precip_total_mm': None, 'rainy_days': None, 'wind_max_kmh': None, 'raw': {}, 'source': 'forecast', 'is_estimate': False, 'reference_years': []}
[{'tool': 'geocode', 'arguments': {'destination': 'Barcelona'}, 'result': {'lat': 41.3825802, 'lon': 2.177073}}, {